# Installing and Importing Libraries

In [1]:
from collections import deque
import numpy as np
import cv2
import telepot
from datetime import datetime
import pytz
from PIL import Image, ImageEnhance
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle, Circle
from mtcnn.mtcnn import MTCNN
from dotenv import load_dotenv
import tensorflow as tf
import time

## Loading Environment Variables

In [2]:
load_dotenv()

True

In [3]:
STORAGE_BUCKET = os.getenv('STORAGE_BUCKET')
BOT_TOKEN = os.getenv('BOT_TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

# Creating Firebase Setup

In [4]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import storage

cred = credentials.Certificate("firebaseKey.json")
firebase_admin.initialize_app(cred, {'storageBucket': STORAGE_BUCKET})  # Initialize once

db = firestore.client() 

# Alert System

## Loading Model

In [5]:
MoBiLSTM_model = tf.keras.models.load_model("detection_model.h5")

In [6]:
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 16
CLASSES_LIST = ["NonViolence", "Violence"]

In [7]:
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    normalized_frame = resized_frame / 255
    return normalized_frame

In [8]:
def img_enhance(frame):
    # Convert OpenCV frame to PIL image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Enhance sharpness
    curr_bri = ImageEnhance.Sharpness(frame_pil)
    new_bri = 1.3
    img_brightened = curr_bri.enhance(new_bri)

    # Enhance color
    curr_col = ImageEnhance.Color(img_brightened)
    new_col = 1.5
    img_col = curr_col.enhance(new_col)

    # Save enhanced image
    img_col.save("finalImage.jpg")

    return img_col


In [9]:
def get_time():
  IST = pytz.timezone('Asia/Kolkata')
  timeNow = datetime.now(IST)
  return timeNow

In [10]:
def draw_faces(filename, result_list):
    # load the image
    data = plt.imread(filename)
    # plot each face as a subplot
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        # define subplot
        plt.subplot(1, len(result_list), i+1)
        plt.axis('off')
        # plot face
        plt.imshow(data[y1:y2, x1:x2])
    # show the plot
    plt.savefig("faces.png")
    plt.show()

In [16]:
def predict_webcam(SEQUENCE_LENGTH, confidence_threshold=0.90):
    cap = cv2.VideoCapture(0)
    bot = telepot.Bot(BOT_TOKEN)
    frames_list = []
    location = "Sector 20, Noida"
    violence_image = 'finalImage.jpg'
    face_image = 'faces.png'
    no_of_detections, alert_sent = 0, 0
    last_alert_time = None

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Preprocess the frame
        normalized_frame = preprocess_frame(frame)
        frames_list.append(normalized_frame)

        # Ensure we have enough frames for the sequence
        if len(frames_list) == SEQUENCE_LENGTH:
            # Perform prediction
            predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_list, axis=0))[0]
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]

            # Display the prediction
            confidence = predicted_labels_probabilities[predicted_label]
            print(f'Predicted: {predicted_class_name}\nConfidence: {confidence}')

            # Display "Violence" in red if confidence is above the threshold
            if predicted_class_name == "Violence" and confidence > confidence_threshold:
                cv2.putText(frame, "Violence", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                no_of_detections += 1

                if no_of_detections >= 20 and alert_sent == 0:
                    current_time = get_time()
                    img_enhance(frame)
                    pixels = plt.imread(violence_image)
                    detector = MTCNN()
                    faces = detector.detect_faces(pixels)
                    draw_faces(violence_image, faces)

                    bot.sendMessage(CHAT_ID, f"VIOLENCE ALERT!! \n at location: {location} \n time: {current_time}")
                    bot.sendPhoto(CHAT_ID, photo=open('finalImage.jpg', 'rb'))
                    bot.sendMessage(CHAT_ID, "Faces Obtained")
                    bot.sendPhoto(CHAT_ID, photo=open('faces.png', 'rb'))

                    storage_client = storage.bucket()

                    try:
                        storage_client.blob(violence_image).upload_from_filename(violence_image)
                        storage_client.blob(face_image).upload_from_filename(face_image)
                    
                        # Get download URLs for the uploaded images
                        violence_image_url = storage_client.blob(violence_image).public_url
                        face_image_url = storage_client.blob(face_image).public_url
                    
                        # Add data to Firestore with download URLs
                        db.collection(location).add({
                            'date': current_time,
                            'image': violence_image_url,
                            'faces': face_image_url
                        })
                    
                    except Exception as e:
                        print(f"Error uploading images to Firebase Storage: {e}")

                    finally:
                        alert_sent = 1
                        last_alert_time = time.time()

            # Check if it's been 5 minutes since the last alert
            if last_alert_time is not None and time.time() - last_alert_time >= 5 * 60:
                alert_sent = 0

            # Clear the frames list for the next sequence
            frames_list = []

        # Display the webcam feed
        cv2.imshow('Violence Detector', frame)

        # Break the loop if 'q' is pressed        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window outside the loop
    cap.release()
    cv2.destroyAllWindows()


In [13]:
# cap = cv2.VideoCapture(0)
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
predict_webcam(SEQUENCE_LENGTH)